#데이터 준비

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_original = pd.read_csv('/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/가공 데이터/추천서비스_최종데이터(10.26).csv',
                          dtype = {'ISBN' : 'str'},
                          encoding = 'utf-8-sig')
df_original.head(2)
df = df_original.copy()

#예제에 사용된 ratings 데이터 분석

In [ ]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/content/drive/MyDrive/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']]
ratings.head(3)

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1


In [ ]:
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

zero_count = 0
for i in range(R_temp.shape[0]):
    for j in range(R_temp.shape[1]):
        if R_temp.values[i][j] == 0:
            zero_count += 1
print(zero_count)

print("0의 비율 : ",zero_count/(R_temp.shape[0] * R_temp.shape[1])*100)

1486126
0의 비율 :  93.69533063577546


In [ ]:
#ratings의 user_id 개수, movie_id 개수, rating 등과 최대한 비슷한 데이터 만들기
print("전체 행 : ", len(ratings))
print("user_id 개수 : ", len(ratings['user_id'].unique()))
print("movie 개수 : ", len(ratings['movie_id'].unique()))

전체 행 :  100000
user_id 개수 :  943
movie 개수 :  1682


#사용자_정보_일관되게_만들기.csv 생성
ratings movie_id 고유값 1600와 같은 ISBN 고유값 설정

In [ ]:
genre = ['소설','시','만화','역사','문학','고전','요리/살림','에세이','여행','과학']

df_genre_isbn = pd.DataFrame({
    'genre' : genre,
    'isbn' : [df[df['genre2'].notna() & df['genre2'].str.contains(g)]['ISBN'].values.tolist() for g in genre]
})
df_genre_isbn

,genre,isbn
0,소설,"[9788937403224 , 9788912090029 , 9788982812057..."
1,시,"[9788936303075 , 9788970638331 , 9788932008318..."
2,만화,"[9788973813438 , 9788974781040 , 9788974781088..."
3,역사,"[9788972783350 , 9788984070202 , 9788971965733..."
4,문학,"[9788973378869 , 9788985467247 , 9788972912101..."
5,고전,"[9788935630271 , 9788974430207 , 9788972912521..."
6,요리/살림,"[9788919020845 , 9788995060711 , 9788990514028..."
7,에세이,"[9788985893473 , 9788985804479 , 9788937403071..."
8,여행,"[9788986903010 , 9788973813483 , 9788977660069..."
9,과학,"[9788984310070 , 9788986128321 , 9788984310100..."


In [ ]:
n=180
for i in range(df_genre_isbn.shape[0]):
  #n보다 list 길이가 길면 뒤에서부터 180개 추출
  print(i, "원래 길이 : ", len(df_genre_isbn.iloc[i,1]) )

  if n < len(df_genre_isbn.iloc[i,1]) :
    df_genre_isbn.iloc[i,1] = df_genre_isbn.iloc[i,1][-n:]

  print(i, "변경 길이 : ", len(df_genre_isbn.iloc[i,1]) )


0 원래 길이 :  4797
0 변경 길이 :  180
1 원래 길이 :  529
1 변경 길이 :  180
2 원래 길이 :  4148
2 변경 길이 :  180
3 원래 길이 :  1077
3 변경 길이 :  180
4 원래 길이 :  2637
4 변경 길이 :  180
5 원래 길이 :  34
5 변경 길이 :  34
6 원래 길이 :  785
6 변경 길이 :  180
7 원래 길이 :  2624
7 변경 길이 :  180
8 원래 길이 :  390
8 변경 길이 :  180
9 원래 길이 :  2214
9 변경 길이 :  180


In [ ]:
sum = 0
for val in df_genre_isbn['isbn']:
  sum = sum+ len(val)
print("총 ISBN unique 값 개수 : ", sum)

df_genre_isbn

총 ISBN unique 값 개수 :  1654


,genre,isbn
0,소설,"[9788934967330 , 9791166687075 , 9791167901477..."
1,시,"[9788954645621 , 9788932030265 , 9788932030319..."
2,만화,"[9791141103606 , 9791138028998 , 9791170622666..."
3,역사,"[9791162201435 , 9788965642015 , 9788972916468..."
4,문학,"[9791192097268 , 9788935673629 , 9791197826917..."
5,고전,"[9788935630271 , 9788974430207 , 9788972912521..."
6,요리/살림,"[9791196334710 , 9791195882885 , 9791187194620..."
7,에세이,"[9791160949643 , 9791190382724 , 9791192107646..."
8,여행,"[9788996333951 , 9788925542850 , 9788991193666..."
9,과학,"[9791167960221 , 9788950991913 , 9791165218386..."


In [ ]:
df_dest = pd.read_csv('/content/drive/MyDrive/설문조사 AI 용 데이터/4. 11_14 isbn 최신꺼 쓰기 모델/사용자_정보_일관되게_만들기_isbn_최신.csv', encoding='utf-8-sig')
df_dest.head(3)

for i in range(df_genre_isbn.shape[0]):
  genre = df_genre_isbn.loc[i,'genre']
  isbn_list = df_genre_isbn.loc[i,'isbn']
  index_list = df_dest[df_dest['장르'] == genre].index


  #df_dest의 index_list의 index의 'ISBN 모음'에 8등분한 isbn_list 차례로 추가
  start = 0
  n = len(isbn_list)//8
  for index in index_list:
    df_dest.loc[index, 'ISBN 모음'] = isbn_list[start:start+n]
    start += n

In [ ]:
df_dest.to_csv('/content/drive/MyDrive/설문조사 AI 용 데이터/4. 11_14 isbn 최신꺼 쓰기 모델/사용자_정보_일관되게_만들기_isbn_최신.csv',
               encoding='utf-8-sig', index=False)


#10만행 데이터 생성

In [ ]:
import pandas as pd
import ast

# CSV 파일을 불러옵니다
new_data_df = pd.read_csv('/content/drive/MyDrive/설문조사 AI 용 데이터/새로운 데이터 만들기.csv', encoding='utf-8-sig')

# 'user_id 범위'와 'ISBN 모음' 열을 리스트로 변환
new_data_df['user_id 범위'] = new_data_df['user_id 범위'].apply(ast.literal_eval)
new_data_df['ISBN 모음'] = new_data_df['ISBN 모음'].apply(ast.literal_eval)

# 확인
new_data_df


,분류,장르,user_id 범위,ISBN 모음
0,10_M,소설,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[9788937403224 , 9788912090029 , 9788982812057..."
1,10_M,시,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]","[9788936303075 , 9788970638331 , 9788932008318..."
2,10_M,만화,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]","[9788973813438 , 9788974781040 , 9788974781088..."
3,10_M,역사,"[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]","[9788972783350 , 9788984070202 , 9788971965733..."
4,10_M,문학,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]","[9788973378869 , 9788985467247 , 9788972912101..."
...,...,...,...,...
75,40_F,고전,"[900, 901, 902, 903, 904, 905, 906, 907, 908, ...","[9788976821034 , 9788971992975 , 9788967350055..."
76,40_F,요리/살림,"[912, 913, 914, 915, 916, 917, 918, 919, 920, ...","[9788950908355 , 9788990991324 , 9788956810478..."
77,40_F,에세이,"[924, 925, 926, 927, 928, 929, 930, 931, 932, ...","[9791196589134 , 9791196814304 , 9788960906037..."
78,40_F,여행,"[936, 937, 938, 939, 940, 941, 942, 943, 944, ...","[9788959371532 , 9788925519159 , 9788984312821..."


In [ ]:
result = pd.DataFrame(columns=['user_id', 'age', 'sex', 'fav_genre', 'isbn', 'rating'])
genre = ['소설','시','만화','역사','문학','고전','요리/살림','에세이','여행','과학']

for i in range(0,100000,2):
  #1. 사용자 : 나이대, 성별, 장르 랜덤 선택
  age = np.random.choice(['10', '20', '30', '40'])
  sex = np.random.choice(['M', 'F'])
  gen = np.random.choice(genre)

  #사용자가 해당하는 분류의 랜덤 user_id 값
  list_temp = new_data_df.loc[(new_data_df['분류'] == age +"_"+ sex) & (new_data_df['장르'] == gen), 'user_id 범위']
  user_id = np.random.choice(list_temp.iloc[0])

  #2. 사용자가 좋아하는 책 저장
  #2-1. 해당 분류의 랜덤 isbn 값
  list_temp = new_data_df.loc[(new_data_df['분류'] == age +"_"+ sex) & (new_data_df['장르'] == gen), 'ISBN 모음']
  isbn = np.random.choice(list_temp.iloc[0])

  #2-2. 높은 rating
  rating = np.random.choice([4,5])

  #2-3. 'user_id', 'age', 'sex', 'isbn', 'rating'순으로 result에 저장
  result.loc[i] = [user_id, age, sex, gen, isbn, rating]


  #3. 사용자가 싫어하는 책 저장
  #3-1 해당하지 않는 분류의 중 랜덤 isbn 값
  list_temp = new_data_df.loc[(new_data_df['분류'] != age +"_"+ sex) | (new_data_df['장르'] == gen), 'ISBN 모음']

  random_index = np.random.choice(list_temp.index)
  random_value = list_temp.loc[random_index]
  no_isbn = np.random.choice(random_value)

  #3-2. 낮은 rating
  rating = np.random.choice([1,2])

  #3-3. 'user_id', 'age', 'sex', 'fav_gen', 'isbn', 'rating'순으로 result에 저장
  result.loc[i+1] = [user_id, age, sex, gen, no_isbn, rating]

  if i % 5000 == 0:
    print(i)



0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000


In [ ]:
result.head(10)

,user_id,age,sex,fav_genre,isbn,rating
0,3,10,M,소설,9788973370801,5
1,3,10,M,소설,9791171240616,2
2,475,20,F,과학,9788983710253,4
3,475,20,F,과학,9791134841386,1
4,194,10,F,요리/살림,9791160320756,5
5,194,10,F,요리/살림,9788990235466,1
6,196,10,F,요리/살림,9788976770905,5
7,196,10,F,요리/살림,9791197693090,1
8,751,40,M,만화,9791170621348,5
9,751,40,M,만화,9788931004021,2


In [ ]:
result.duplicated(subset=['user_id','isbn']).sum()

34227

In [ ]:
#user_id, ISBN 중복 제거
result = result.drop_duplicates(subset = ['user_id', 'isbn'], keep = 'first')
result.reset_index(drop = True, inplace = True)

#result 행 전체 섞기
result = result.sample(frac=1).reset_index(drop=True)
result.to_csv('/content/drive/MyDrive/설문조사 AI 용 데이터/10만행_데이터.csv', encoding='utf-8-sig', index=False)

#고민 수정한 새로운 데이터

In [ ]:
import pandas as pd
import ast

# CSV 파일을 불러옵니다
new_data_df = pd.read_csv('/content/drive/MyDrive/설문조사 AI 용 데이터/2. 새로운_모델_11-14 완성/사용자_정보_일관되게_만들기.csv', encoding='utf-8-sig')

# 'user_id 범위'와 'ISBN 모음' 열을 리스트로 변환
new_data_df['user_id 범위'] = new_data_df['user_id 범위'].apply(ast.literal_eval)
new_data_df['ISBN 모음'] = new_data_df['ISBN 모음'].apply(ast.literal_eval)

# 확인
new_data_df

,분류,장르,user_id 범위,ISBN 모음
0,10_M,소설,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[9788937403224 , 9788912090029 , 9788982812057..."
1,10_M,시,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]","[9788936303075 , 9788970638331 , 9788932008318..."
2,10_M,만화,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]","[9788973813438 , 9788974781040 , 9788974781088..."
3,10_M,역사,"[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]","[9788972783350 , 9788984070202 , 9788971965733..."
4,10_M,문학,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]","[9788973378869 , 9788985467247 , 9788972912101..."
...,...,...,...,...
75,40_F,고전,"[900, 901, 902, 903, 904, 905, 906, 907, 908, ...","[9788976821034 , 9788971992975 , 9788967350055..."
76,40_F,요리/살림,"[912, 913, 914, 915, 916, 917, 918, 919, 920, ...","[9788950908355 , 9788990991324 , 9788956810478..."
77,40_F,에세이,"[924, 925, 926, 927, 928, 929, 930, 931, 932, ...","[9791196589134 , 9791196814304 , 9788960906037..."
78,40_F,여행,"[936, 937, 938, 939, 940, 941, 942, 943, 944, ...","[9788959371532 , 9788925519159 , 9788984312821..."


In [ ]:
result = pd.DataFrame(columns=['user_id', 'age', 'sex', 'fav_genre', 'isbn', 'rating'])
genre = ['소설','시','만화','역사','문학','고전','요리/살림','에세이','여행','과학']

user_id_age = {}

for i in range(0,140000,2):
  #1. user_id : 사용자 id 0부터 959 까지 랜덤 선택하고 '새로운데이터만들기'에서 해당 행 가져오기
  user_id = np.random.randint(0,960)
  filtered_df = new_data_df[new_data_df['user_id 범위'].apply(lambda x: user_id in x)]

  #2. 기본정보 : 나이, 성별, 장르 셋팅
  sg = filtered_df.iloc[0]['분류']
  age_dae, sex= sg.split('_')
  gen = filtered_df.iloc[0]['장르']

  #user_id_age 딕셔너리에 user_id가 없으면 나이 값 초기화하고 저장
  if user_id not in user_id_age.keys():
    age = int(age_dae) + np.random.randint(0,10)
    user_id_age[user_id] = age
  else:
    age = user_id_age[user_id]

  #3. 좋아하는 책 : 별점 4-5
  list_temp = filtered_df['ISBN 모음'].iloc[0]
  isbn = np.random.choice(list_temp)
  rating = np.random.choice([4,5])

  result.loc[i] = [user_id, age, sex, gen, isbn, rating]

  #4. 싫어하는 책 : 별점 1-2
  #3-1 해당하지 않는 분류의 중 랜덤 isbn 값
  filtered_df_neg = new_data_df[new_data_df['user_id 범위'].apply(lambda x: user_id not in x)]

  list_temp = filtered_df_neg['ISBN 모음'].iloc[0]
  neg_isbn = np.random.choice(list_temp)
  neg_rating = np.random.choice([1,2])

  result.loc[i+1] = [user_id, age, sex, gen, neg_isbn, neg_rating]

  if i % 5000 == 0:
    print(i)



0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000


In [ ]:
result

,user_id,age,sex,fav_genre,isbn,rating
0,254,21,M,시,9788932042787,4
1,254,21,M,시,9788973370771,2
2,475,20,F,과학,9788984050051,5
3,475,20,F,과학,9788973370801,2
4,936,40,F,여행,9788961885775,4
...,...,...,...,...,...,...
139995,570,39,M,에세이,9788932308890,2
139996,440,28,F,요리/살림,9788983754349,5
139997,440,28,F,요리/살림,9788932900209,2
139998,618,31,F,시,9791141606831,5


In [ ]:
import pandas as pd

# 예시 데이터가 저장된 result 데이터프레임

# user_id별로 그룹화하여 각 컬럼의 고유값 수(count) 확인
consistency_check = result.groupby('user_id').agg({
    'age': 'nunique',
    'sex': 'nunique',
    'fav_genre': 'nunique'
}).reset_index()

# 일관되지 않은 user_id 찾기 (고유값이 1보다 큰 경우)
inconsistent_users = consistency_check[(consistency_check['age'] > 1) |
                                       (consistency_check['sex'] > 1) |
                                       (consistency_check['fav_genre'] > 1)]

# 결과 출력
if not inconsistent_users.empty:
    print("일관되지 않은 user_id가 존재합니다.")
    print(inconsistent_users)
else:
    print("모든 user_id에 대해 age, sex, fav_genre 값이 일관됩니다.")


모든 user_id에 대해 age, sex, fav_genre 값이 일관됩니다.


In [ ]:
result.duplicated(subset=['user_id','isbn']).sum()

100933

In [ ]:
#user_id, ISBN 중복 제거
result = result.drop_duplicates(subset = ['user_id', 'isbn'], keep = 'first')
result.reset_index(drop = True, inplace = True)

#result 행 전체 섞기
result = result.sample(frac=1).reset_index(drop=True)
result.to_csv('/content/drive/MyDrive/설문조사 AI 용 데이터/새로운 모델 11-14 완성/10만행_데이터_user_정보일관.csv', encoding='utf-8-sig', index=False)

#ISBN 바꾸기 최신거로

In [ ]:
import pandas as pd
import ast

# CSV 파일을 불러옵니다
new_data_df = pd.read_csv('/content/drive/MyDrive/설문조사 AI 용 데이터/4. 11_14 isbn 최신꺼 쓰기 모델/사용자_정보_일관되게_만들기_isbn_최신.csv', encoding='utf-8-sig')

# 'user_id 범위'와 'ISBN 모음' 열을 리스트로 변환
new_data_df['user_id 범위'] = new_data_df['user_id 범위'].apply(ast.literal_eval)
new_data_df['ISBN 모음'] = new_data_df['ISBN 모음'].apply(ast.literal_eval)

# 확인
new_data_df


,분류,장르,user_id 범위,ISBN 모음
0,10_M,소설,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[9788934967330 , 9791166687075 , 9791167901477..."
1,10_M,시,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]","[9788954645621 , 9788932030265 , 9788932030319..."
2,10_M,만화,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]","[9791141103606 , 9791138028998 , 9791170622666..."
3,10_M,역사,"[36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]","[9791162201435 , 9788965642015 , 9788972916468..."
4,10_M,문학,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]","[9791192097268 , 9788935673629 , 9791197826917..."
...,...,...,...,...
75,40_F,고전,"[900, 901, 902, 903, 904, 905, 906, 907, 908, ...","[9788976821034 , 9788971992975 , 9788967350055..."
76,40_F,요리/살림,"[912, 913, 914, 915, 916, 917, 918, 919, 920, ...","[9791188926053 , 9788965181750 , 9791191994162..."
77,40_F,에세이,"[924, 925, 926, 927, 928, 929, 930, 931, 932, ...","[9791188605279 , 9791170402527 , 9791190999175..."
78,40_F,여행,"[936, 937, 938, 939, 940, 941, 942, 943, 944, ...","[9791189938482 , 9791189510046 , 9791196601904..."


In [ ]:
result = pd.DataFrame(columns=['user_id', 'age', 'sex', 'fav_genre', 'isbn', 'rating'])
genre = ['소설','시','만화','역사','문학','고전','요리/살림','에세이','여행','과학']

for i in range(0,100000,2):
  #1. 사용자 : 나이대, 성별, 장르 랜덤 선택
  age = np.random.choice(['10', '20', '30', '40'])
  sex = np.random.choice(['M', 'F'])
  gen = np.random.choice(genre)

  #사용자가 해당하는 분류의 랜덤 user_id 값
  list_temp = new_data_df.loc[(new_data_df['분류'] == age +"_"+ sex) & (new_data_df['장르'] == gen), 'user_id 범위']
  user_id = np.random.choice(list_temp.iloc[0])

  #2. 사용자가 좋아하는 책 저장
  #2-1. 해당 분류의 랜덤 isbn 값
  list_temp = new_data_df.loc[(new_data_df['분류'] == age +"_"+ sex) & (new_data_df['장르'] == gen), 'ISBN 모음']
  isbn = np.random.choice(list_temp.iloc[0])

  #2-2. 높은 rating
  rating = np.random.choice([4,5])

  #2-3. 'user_id', 'age', 'sex', 'isbn', 'rating'순으로 result에 저장
  result.loc[i] = [user_id, age, sex, gen, isbn, rating]


  #3. 사용자가 싫어하는 책 저장
  #3-1 해당하지 않는 분류의 중 랜덤 isbn 값
  list_temp = new_data_df.loc[(new_data_df['분류'] != age +"_"+ sex) | (new_data_df['장르'] == gen), 'ISBN 모음']

  random_index = np.random.choice(list_temp.index)
  random_value = list_temp.loc[random_index]
  no_isbn = np.random.choice(random_value)

  #3-2. 낮은 rating
  rating = np.random.choice([1,2])

  #3-3. 'user_id', 'age', 'sex', 'fav_gen', 'isbn', 'rating'순으로 result에 저장
  result.loc[i+1] = [user_id, age, sex, gen, no_isbn, rating]

  if i % 5000 == 0:
    print(i)



0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000


In [ ]:
result.head(10)

,user_id,age,sex,fav_genre,isbn,rating
0,703,30,F,여행,9791187345381,4
1,703,30,F,여행,9788972912521,1
2,252,20,M,시,9788972759591,5
3,252,20,M,시,9791169518352,2
4,280,20,M,역사,9788997066438,4
5,280,20,M,역사,9791130699691,2
6,605,30,F,소설,9791192313412,4
7,605,30,F,소설,9791188501380,2
8,814,40,M,에세이,9791171251179,4
9,814,40,M,에세이,9791198289445,2


In [ ]:
result.duplicated(subset=['user_id','isbn']).sum()

34256

In [ ]:
#user_id, ISBN 중복 제거
result = result.drop_duplicates(subset = ['user_id', 'isbn'], keep = 'first')
result.reset_index(drop = True, inplace = True)

#result 행 전체 섞기
result = result.sample(frac=1).reset_index(drop=True)
result.to_csv('/content/drive/MyDrive/설문조사 AI 용 데이터/4. 11_14 isbn 최신꺼 쓰기 모델/10만행_데이터_isbn_갱신.csv', encoding='utf-8-sig', index=False)